In [1]:
from voice import *
from voiceEN import *

C:\Users\shiha\anaconda3\envs\lang\Lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
C:\Users\shiha\AppData\Roaming\Python\Python311\site-packages\TTS\api.py:77: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")
C:\Users\shiha\anaconda3\envs\lang\Lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [25]:
import moviepy.editor as mpy
from moviepy.audio.AudioClip import AudioArrayClip

def audio_fadein(clips,fade_in = 0.05):
    final_clips = [
        clip.audio_fadein(fade_in).audio_fadeout(fade_in)
        for clip in clips
    ]
    return(final_clips)

def text2audio(text:str, lang:str, audio = "./cache/audio/1.wav"):
    #audio from text
    text = text.replace('\r', ' ').replace('\n', ' ')
    if lang == "en":
        tts.tts_to_file(text=text, file_path=audio)
    else:
        o3, o4 = tts_fn(text, lang = lang)
        from scipy.io.wavfile import write
        write(audio,o4[0],o4[1])
    return()

def text2video(text:str, lang:str, title = "      ", folder = "./cache/", prefix = "1", img = ""):
    #make audio
    o4 = text2audio(text, lang = lang, audio = folder+"audio/"+prefix+".wav");
    audio = mpy.AudioFileClip(folder+"audio/"+prefix+".wav")
    #audio = AudioArrayClip(o4[1],fps = o4[0])
    #make text
    if lang == "cn":
        txt_clip  = mpy.TextClip(text, fontsize = 10, color = "white", font = "Microsoft-YaHei-&-Microsoft-YaHei-UI")
    elif lang == "jp":
        txt_clip  = mpy.TextClip(text, fontsize = 10, color = "white", font = "wqy-microhei.ttc")
    else:
        txt_clip  = mpy.TextClip(text, fontsize = 10, color = "white")
    txt_clip = txt_clip.set_pos('center').set_duration(audio.duration)
    title_clip  = mpy.TextClip(title, fontsize = 40, color = "black")
    title_clip = title_clip.set_pos('top').set_duration(audio.duration)
    #make video
    clip = mpy.ImageClip(img)
    clip = clip.set_duration(audio.duration)
    clip.audio = audio
    clip  = mpy.CompositeVideoClip([clip, txt_clip, title_clip])
    clip.fps = 12
    #clip.write_videofile("./cache/clip.mp4");
    return(clip)

def json2video(json:list, lang = "cn", output = "./cache/video.mp4"):
    videos = []
    count = 0;
    for l in json:
        if l['cg'] != "":
            #get cg
            import wget
            img = "./cache/img"+str(count+1)+".png"
            wget.download(l['cg'], out=img)
            #split text
            text = l['summary']
            split = text.replace('。','.').split('.')
            clips = []
            for s in split:
                if len(s) > 5:
                    count = count + 1;
                    clips.append(text2video(text = s, lang = lang, title = l['stage'], prefix = str(count), img = img))
            clips = audio_fadein(clips)
            videos.append(mpy.concatenate_videoclips(clips, method="compose"))
    videos = audio_fadein(videos)
    concat_clip = mpy.concatenate_videoclips(videos, method="compose")
    concat_clip.write_videofile(output)

#output chapter content to html
def run(query:str, output = "./cache/video.mp4", path="../db/cndb", lang = "cn"):
    import chromadb
    client = chromadb.PersistentClient(path=path)
    db = client.get_collection("langchain")
    data = db.get()
    tmp = []
    for k in range(len(data["ids"])):
        if query in data["metadatas"][k]["stage"]:
            this = {"stage": data["metadatas"][k]["stage"], 
                    "summary": data["documents"][k],
                    "cg": data["metadatas"][k]["cg"],
                   }
            tmp.append(this);
    json2video(json = tmp, lang = lang, output = output)

import sys, os
# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

import logging
logger = logging.getLogger('my-logger')
logger.propagate = False

In [ ]:
run("ZT-", lang = "jp", path="../db/jpdb", output = "./cache/ZT_JP.mp4")

t:  46%|█████████████████████████████▎                                  | 5576/12193 [05:22<06:15, 17.61it/s, now=None]


In [6]:
run("13-22", lang = "en",  path="../db/arkdb", output = "./cache/13-22EN.mp4")

In [20]:
run("13-22", lang = "jp",  path="../db/jpdb", output = "./cache/13-22JP.mp4")

t:  25%|████████████████▌                                                 | 714/2835 [03:55<01:45, 20.12it/s, now=None]